In [ ]:
from omegaconf import OmegaConf
from datasets import load_data
import numpy as np
config = OmegaConf.load("./configs/default_config.yaml")

for dataset in ['Imagenette', 'Flowers', 
                'EuroSAT', 'Aircraft',
                'CIFAR10', 'CIFAR100',
                'StanfordCars', 'OxfordIIITPet', 
                'Caltech101', 'DTD' ]:
    
    config.dataset = dataset
    train_dataset, val_dataset, test_dataset, N, train_transform, transform = load_data(False, config) 
    image, label = val_dataset[0]  # Extract the image and label
    pixels = np.array(image)
    print(dataset, len(train_dataset), len(val_dataset), len(test_dataset), min(pixels[0][0]),  max(pixels[0][0]), N)

datadir /home/mheuillet/Desktop/robust_training/data
Imagenette 9469 982 2943 39 39 10
Flowers 1020 1020 6149 61 128 102
EuroSAT 18360 3240 5400 138 208 10
Aircraft 3334 3333 3333 111 186 100
Files already downloaded and verified
CIFAR10 47500 2500 10000 121 194 10
CIFAR100 47500 2500 10000 10 52 100


In [7]:
from torchvision import datasets

datasets.Caltech101(root='./data',  download=True, )

Downloading...
From (original): https://drive.google.com/uc?id=137RyRjvTBkBiIfeYBNZBtViDHQ6_Ewsp
From (redirected): https://drive.usercontent.google.com/download?id=137RyRjvTBkBiIfeYBNZBtViDHQ6_Ewsp&confirm=t&uuid=67e0f9af-282a-47e5-bf69-61f6d0cff3d0
To: /home/mheuillet/Desktop/robust_training/data/caltech101/101_ObjectCategories.tar.gz
100%|██████████| 132M/132M [00:21<00:00, 6.06MB/s] 


Extracting ./data/caltech101/101_ObjectCategories.tar.gz to ./data/caltech101


Downloading...
From: https://drive.google.com/uc?id=175kQy3UsZ0wUEHZjqkUDdNVssr7bgh_m
To: /home/mheuillet/Desktop/robust_training/data/caltech101/Annotations.tar
100%|██████████| 14.0M/14.0M [00:03<00:00, 3.71MB/s]


Extracting ./data/caltech101/Annotations.tar to ./data/caltech101


Dataset Caltech101
    Number of datapoints: 8677
    Root location: ./data/caltech101
    Target type: ['category']

In [5]:
x[0]

(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=510x337>, 0)